In [1]:
!pip install pytorch-lightning
!pip install torch_pruning
!rm -r matteobarato*
!git clone https://github.com/matteobarato/matteobarato-KD_ObjectPose6D.git

rm: cannot remove 'matteobarato*': No such file or directory
Cloning into 'matteobarato-KD_ObjectPose6D'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 73 (delta 36), reused 64 (delta 27), pack-reused 0
Unpacking objects: 100% (73/73), 29.16 KiB | 355.00 KiB/s, done.


In [2]:
# Downalod ResNet18 weights
# !wget -nc http://cipizio.it/storage/Nas-ResNet/resnet18_net_e_199.pth
# !mkdir checkpoint
# !mv resnet18_net_e_199.pth ./checkpoint

# Download trained student with gates and stricsigmoid activation on 30 epochs with ADAM
!#wget -nc http://cipizio.it/storage/Nas-ResNet/student_e50_ADAM_stricsig_lre-2_lp_e-4_dw_8.pth
#!mkdir checkpoint
#!mv student_e50_ADAM_stricsig_lre-2_lp_e-4_dw_8.pth ./checkpoint

In [3]:
'''Train CIFAR10 with PyTorch.'''
import sys
sys.path.append("./matteobarato-KD_ObjectPose6D")
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
import src.utils as utils
from src.KD import GatedKD
from src.models import ResNet18
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


import torch_pruning as tp

import os
import argparse

In [4]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
args = {'resume':False, 'lr':0.001, 'lambda_gating':0.1}
seed_everything(42, workers=True)


42

## Dataset CIFAR10

In [5]:
# Data
torch.manual_seed(43)
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])




trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

val_size = 5000
train_size = len(trainset) - val_size

trainset, validationset = torch.utils.data.random_split(trainset, [train_size, val_size])

train_dataloaders = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
val_dataloaders = torch.utils.data.DataLoader(
    validationset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
test_dataloaders = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Model

In [27]:
args['lr'] = 1e-2
args['lambda_penalty'] = 1e-5
args['distill_weight'] = 0.9

teacher_model = ResNet18(feature_extracting=False, remove_avg_pool_layer=True, lr=5e-3, optim='adam', scheduler_t_max=300)
student_model = ResNet18(lr=args['lr'], optim='adam', scheduler_t_max=100, use_gates_with_penalty=args['lambda_penalty'])

a = student_model.apply_gates()

## Train Teacher

In [28]:
teacher_trainer = Trainer(deterministic=True, accelerator='gpu', devices=1, callbacks=[EarlyStopping(monitor="val_loss", mode="min")],  min_epochs=100, max_epochs=300)
teacher_trainer.fit(teacher_model, train_dataloaders, val_dataloaders)
kd_trainer.test(dataloaders=test_dataloaders)

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

NameError: name 'kd_trainer' is not defined

In [33]:
!zip -r lightning_logs.zip ./lightning_logs

  adding: lightning_logs/ (stored 0%)
  adding: lightning_logs/version_9/ (stored 0%)
  adding: lightning_logs/version_9/events.out.tfevents.1656412439.0714603c8330.33.9 (deflated 67%)
  adding: lightning_logs/version_9/checkpoints/ (stored 0%)
  adding: lightning_logs/version_9/checkpoints/epoch=22-step=8096.ckpt (deflated 17%)
  adding: lightning_logs/version_9/hparams.yaml (stored 0%)


## Training Student KD

In [ ]:
distiller_compr = GatedKD(teacher_model, student_model, train_dataloaders, test_dataloaders, loss_fn=nn.KLDivLoss(), 
distill_weight=args['distill_weight'], temp=20)  

In [ ]:
kd_compr_trainer = Trainer(deterministic=True, accelerator='gpu', devices=1, callbacks=[EarlyStopping(monitor="gates_loss", mode="min")],  min_epochs=30, max_epochs=100)
kd_compr_trainer.fit(distiller_compr, train_dataloaders, val_dataloaders)

In [ ]:
kd_compr_trainer.test(dataloaders=test_dataloaders)

In [ ]:
use_mean = False
#NAS.gating_threshold = 1e-2
student_model.NAS.verbose=True
student_model.estimate_required_channels(use_mean=use_mean)   
student_model.NAS.verbose=False
student_model.optimize(use_mean=use_mean, amount=0.4)
print("Improvment model size:", distiller.calc_improvement())

In [ ]:
print(student_model)

In [ ]:
%%time
kd_trainer.test(dataloaders=test_dataloaders)